Parse table from Wikipedia webpage

In [ ]:
import pandas as pd # library for data analysis
from bs4 import BeautifulSoup # library to parse web pages
import requests # library to handle requests
import csv

Parse the website with BeautifulSoup - extracting the data from the table

In [ ]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')
#print(table_body)

# get the headers of the table and store in a list
table_headers = []
headers = table_body.find_all('th')
for header in headers:
    header_value = header.get_text().strip()
    table_headers.append(header_value)

# get the rows of the table
rows = table_body.find_all('tr')
for row in rows:
    row_data = {}
    cells = row.find_all('td')
    for position, cell in enumerate(cells):
        value = cell.get_text().strip()
        key = table_headers[position]
        # add the value to a dictionary
        row_data[key] = value

    # check that there is some data and that Borough is not unassigned
    if row_data and row_data.get('Borough', '') != 'Not assigned':
        data.append(row_data)



load the data into a DataFrame

In [ ]:
df = pd.DataFrame(data)
# rename the postal code heading
df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df

In [ ]:
# print the shape of the data
df.shape

In [ ]:
#import geocoder # import geocoder
coordinate_data = {}
with open('Geospatial_Coordinates.csv') as in_file:
    data = csv.DictReader(in_file)
    for row in data:
        coordinate_data[row['Postal Code']] = {'longitude': row['Longitude'],
                                               'latitude': row['Latitude']}

def get_coordinates(postal_code):
    ret = coordinate_data.get(postal_code, {})
    latitude = ret.get('latitude')
    longitude = ret.get('longitude')
    return longitude, latitude

get the longitude and latitude for each postcode



In [ ]:
longitude = []
latitude = []

for index, row in df.iterrows():
    postal_code = row.get('PostalCode')
    row_long, row_lat = get_coordinates(postal_code=postal_code)
    longitude.append(row_long)
    latitude.append(row_lat)

df['Latitude'] = latitude
df['Longitude'] = longitude

In [ ]:
df

